In [1]:
import os
import sys
from time import gmtime, strftime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import torch
import poutyne
from poutyne import Model,Experiment

In [3]:
from data.custom_data import filepath_dataframe,nucPaired_fpDataframe
from data.selection import Selection,SelectionSet_1
from data.transformation import Transform_CnnLstmS,Transform_CnnS
from data.torchData import DataLoadings,DataLoading,PairDataLoading

from data.custom_data import nucPaired_fpDataframe
from data.torchData import PairDataLoading,DataLoading
from training.contrastive_pretraining import Contrastive_PreTraining
from training.finetuning import FineTuneCNN

import models

In [4]:
#####################################################################################################################

# random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# gpu setting
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
device = DEVICE

## data directory
data_dir  = 'E:\\external_data\\opera_csi\\Session_2\\experiment_data\\experiment_data\\exp_7_amp_spec_only\\npy_format'
readtype = 'npy'
splitchar = '\\'
fpath = '.'

# data selection
data_selection = SelectionSet_1() # Selection(split='random',test_sub=0.2,val_sub=0.1,nuc='NUC1',room=1,sample_per_class=None)
dataselection_name = 'SelectionSet1' # 'EXP7-NUC1-Room1-Amp-RandomSplit-ResReduced'

# data loading
transform = None
batch_size = 128
num_workers = 0

# training
optimizer_builder = torch.optim.SGD
lr = 0.0005
pretrain_epochs = 800
finetune_epochs = 200
supervision = None
temperature = 0.5

# model
builder = lambda : (models.baseline.ConvNet1D(strides=[16,8,4]), 6144)
hidden_layer = 128
network_name = 'ConvNet1D'

# Experiment Name
comment = '3LayerDefault'
exp_name = f'{network_name}_NTXent_{dataselection_name}_Comment-{comment}'

# auto
record_dir = os.path.join(fpath,'record')
model_dir = os.path.join(fpath,'saved_model')
model_fname = os.path.join(model_dir,f'Encoder___{exp_name}')
print('Experiment Name: ',exp_name)
print('Cuda Availability: ',torch.cuda.is_available())

Experiment Name:  ConvNet1D_NTXent_SelectionSet1_Comment-3LayerDefault
Cuda Availability:  True


In [5]:
# -----------------------------------Main-------------------------------------------

# if __name__ == '__main__':

# data preparation
df = filepath_dataframe(data_dir,splitchar)
df = nucPaired_fpDataframe(df)
df_train,df_val,df_test = data_selection(df)
df_train = pd.concat([df_train,df_val])

In [6]:
pretrain_loading = PairDataLoading(transform=transform,
                                   batch_size=batch_size,
                                   readtype=readtype,
                                   supervision=True,
                                   num_workers=num_workers,
                                   drop_last=True)


pretrain_loader = pretrain_loading(df_train)

In [7]:
clr = Contrastive_PreTraining(encoder_builder=builder,
                              batch_size=batch_size,
                              supervision=supervision,
                              temperature=temperature
                              )

encoder = clr.train(train_loader=pretrain_loader,
                    epochs=pretrain_epochs,
                    rtn_history=False,
                    device=device)

torch.save(encoder.state_dict(),model_fname)
del encoder, clr, pretrain_loader

Epoch 1 >>>>>>>>>>>> loss: 2.7818081378936768
Epoch 2 >>>>>>>>>>>> loss: 2.7762794494628906
Epoch 3 >>>>>>>>>>>> loss: 2.7711148262023926
Epoch 4 >>>>>>>>>>>> loss: 2.7711105346679688
Epoch 5 >>>>>>>>>>>> loss: 2.7704267501831055
Epoch 6 >>>>>>>>>>>> loss: 2.7703018188476562
Epoch 7 >>>>>>>>>>>> loss: 2.7699735164642334
Epoch 8 >>>>>>>>>>>> loss: 2.769225835800171
Epoch 9 >>>>>>>>>>>> loss: 2.768512487411499
Epoch 10 >>>>>>>>>>>> loss: 2.76670503616333
Epoch 11 >>>>>>>>>>>> loss: 2.7694859504699707
Epoch 12 >>>>>>>>>>>> loss: 2.7599620819091797
Epoch 13 >>>>>>>>>>>> loss: 2.7359790802001953
Epoch 14 >>>>>>>>>>>> loss: 2.7215702533721924
Epoch 15 >>>>>>>>>>>> loss: 2.6975810527801514
Epoch 16 >>>>>>>>>>>> loss: 2.7277119159698486
Epoch 17 >>>>>>>>>>>> loss: 2.6771163940429688
Epoch 18 >>>>>>>>>>>> loss: 2.681680202484131
Epoch 19 >>>>>>>>>>>> loss: 2.6459248065948486
Epoch 20 >>>>>>>>>>>> loss: 2.6554291248321533
Epoch 21 >>>>>>>>>>>> loss: 2.6789443492889404
Epoch 22 >>>>>>>>>>>> loss:

Epoch 176 >>>>>>>>>>>> loss: 1.9515788555145264
Epoch 177 >>>>>>>>>>>> loss: 1.9466173648834229
Epoch 178 >>>>>>>>>>>> loss: 1.9599907398223877
Epoch 179 >>>>>>>>>>>> loss: 1.971801519393921
Epoch 180 >>>>>>>>>>>> loss: 1.9512381553649902
Epoch 181 >>>>>>>>>>>> loss: 1.973233938217163
Epoch 182 >>>>>>>>>>>> loss: 1.9542412757873535
Epoch 183 >>>>>>>>>>>> loss: 1.9645557403564453
Epoch 184 >>>>>>>>>>>> loss: 1.962028980255127
Epoch 185 >>>>>>>>>>>> loss: 1.9458078145980835
Epoch 186 >>>>>>>>>>>> loss: 1.9521450996398926
Epoch 187 >>>>>>>>>>>> loss: 1.9478600025177002
Epoch 188 >>>>>>>>>>>> loss: 1.9446121454238892
Epoch 189 >>>>>>>>>>>> loss: 1.9481627941131592
Epoch 190 >>>>>>>>>>>> loss: 1.9548943042755127
Epoch 191 >>>>>>>>>>>> loss: 1.9440782070159912
Epoch 192 >>>>>>>>>>>> loss: 1.9608967304229736
Epoch 193 >>>>>>>>>>>> loss: 1.955028772354126
Epoch 194 >>>>>>>>>>>> loss: 1.9521421194076538
Epoch 195 >>>>>>>>>>>> loss: 1.9596457481384277
Epoch 196 >>>>>>>>>>>> loss: 1.9672431945800

KeyboardInterrupt: 

In [ ]:
##### FINE-TUNING #####

# data loading
data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                           num_workers=num_workers,drop_last=True)
test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                           num_workers=num_workers,drop_last=True)

df_train = df_train.rename({'fullpath_x':'fullpath'},axis=1)
df_val = df_val.rename({'fullpath_x':'fullpath'},axis=1)
df_test = df_test.rename({'fullpath_x':'fullpath'},axis=1)

train_loader = data_loading(df_train)
val_loader   = data_loading(df_val)
test_loader  = test_loading(df_test)

# load and create model
model = FineTuneCNN(model_path=model_fname,
                    encoder_builder=builder,
                    hidden_layer=hidden_layer,
                    n_classes=df.activity.nunique())

In [ ]:
# train with poutyne
mdl = Model(model,'adam','cross_entropy',
            batch_metrics=['accuracy'],
            epoch_metrics=[poutyne.F1('micro'),poutyne.F1('macro')]).to(device)
history = mdl.fit_generator(train_generator=train_loader,valid_generator=test_loader,epochs=finetune_epochs)